In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForQuestionAnswering
import time

In [3]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [4]:
# Define the path to the SQuAD 2.0 training data
path = "/kaggle/input/squad-dataset-mee/train-v2.0.json"

# Load and preprocess the SQuAD 2.0 data
def load_squad_data(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    texts = []
    questions = []
    answers = []

    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    texts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return texts[0:10850], questions[0:10850], answers[0:10850]

# Preprocess the data to find answer start and end positions
train_texts, train_queries, train_answers = load_squad_data(path)

In [5]:

# Give the path for SQuAD 2.0 validation data
path = Path('/kaggle/input/squad-dataset-mee/dev-v2.0.json')

# Load and preprocess the SQuAD 2.0 data
def load_squad_data(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    texts = []
    questions = []
    answers = []

    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    texts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return texts[0:2537], questions[0:2537], answers[0:2537]

# Preprocess the data to find answer start and end positions
val_texts, val_queries, val_answers = load_squad_data(path)

In [6]:
for answer, text in zip(train_answers, train_texts):
    real_answer = answer['text']
    start_idx = answer['answer_start']
    # Get the real end index
    end_idx = start_idx + len(real_answer)

    # Deal with the problem of 1 or 2 more characters
    if text[start_idx:end_idx] == real_answer:
        answer['answer_end'] = end_idx
    # When the real answer is more by one character
    elif text[start_idx-1:end_idx-1] == real_answer:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1
    # When the real answer is more by two characters
    elif text[start_idx-2:end_idx-2] == real_answer:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2

In [7]:

for answer, text in zip(val_answers, val_texts):
    real_answer = answer['text']
    start_idx = answer['answer_start']
    # Get the real end index
    end_idx = start_idx + len(real_answer)

    # Deal with the problem of 1 or 2 more characters
    if text[start_idx:end_idx] == real_answer:
        answer['answer_end'] = end_idx
    # When the real answer is more by one character
    elif text[start_idx-1:end_idx-1] == real_answer:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1
    # When the real answer is more by two characters
    elif text[start_idx-2:end_idx-2] == real_answer:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [18]:
train_encodings = tokenizer(train_texts, train_queries, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, val_queries, truncation=True, padding=True)
# If necessary (based on the maximum length), truncate both text and query sequences to fit the specified length.

In [25]:
train_encodings['input_ids'][0].count(102)

2

In [15]:
print("Passage: ",train_texts[0])
print("Query: ",train_queries[0])
print("Answer: ",train_answers[0])


Passage:  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Query:  When did Beyonce start becoming popular?
Answer:  {'text': 'in the late 1990s', 'answer_start': 269, 'answer_end': 286}


In [26]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    
    count = 0
    
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))
        
        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
            
        # if end position is None, the 'char_to_token' function points to the space after the correct token, so add - 1
        if end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - 1)
            
            # if end position is still None the answer passage has been truncated
            if end_positions[-1] is None:
                count += 1
                end_positions[-1] = tokenizer.model_max_length
    print(count)
    
    # Update the data in dictionary
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

2
0


In [27]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)


In [28]:
train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [29]:
(train_dataset[0])

{'input_ids': tensor([  101, 20773, 21025, 19358, 22815,  1011,  5708,  1006,  1013, 12170,
         23432, 29715,  3501, 29678, 12325, 29685,  1013, 10506,  1011, 10930,
          2078,  1011,  2360,  1007,  1006,  2141,  2244,  1018,  1010,  3261,
          1007,  2003,  2019,  2137,  3220,  1010,  6009,  1010,  2501,  3135,
          1998,  3883,  1012,  2141,  1998,  2992,  1999,  5395,  1010,  3146,
          1010,  2016,  2864,  1999,  2536,  4823,  1998,  5613,  6479,  2004,
          1037,  2775,  1010,  1998,  3123,  2000,  4476,  1999,  1996,  2397,
          4134,  2004,  2599,  3220,  1997,  1054,  1004,  1038,  2611,  1011,
          2177, 10461,  1005,  1055,  2775,  1012,  3266,  2011,  2014,  2269,
          1010, 25436, 22815,  1010,  1996,  2177,  2150,  2028,  1997,  1996,
          2088,  1005,  1055,  2190,  1011,  4855,  2611,  2967,  1997,  2035,
          2051,  1012,  2037, 14221,  2387,  1996,  2713,  1997, 20773,  1005,
          1055,  2834,  2201,  1010, 20

In [30]:
train_dataset[0]["start_positions"]

tensor(67)

In [16]:
print(train_dataset)

In [31]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [32]:
for batch_idx, batch in enumerate(train_loader):
    if batch_idx == 0:
        print(batch)
        break

{'input_ids': tensor([[ 101, 1999, 2054,  ...,    0,    0,    0],
        [ 101, 1996, 2682,  ...,    0,    0,    0],
        [ 101, 9829, 3742,  ...,    0,    0,    0],
        ...,
        [ 101, 1996, 8720,  ...,    0,    0,    0],
        [ 101, 2006, 2238,  ...,    0,    0,    0],
        [ 101, 1037, 2969,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'start_positions': tensor([101,   7,  52, 159,  56,   7,  62, 114]), 'end_positions': tensor([105,   7,  69, 159,  60,   7,  68, 114])}


In [18]:
input_ids = torch.tensor(train_encodings['input_ids']).unsqueeze(0)
segment_ids = torch.tensor(train_encodings['token_type_ids']).unsqueeze(0)

In [34]:
import torch
import torch.optim as optim
import time
from transformers import BertForQuestionAnswering, BertTokenizer

# Initialize BERT model and tokenizer
model_name = 'bert-base-uncased'
model = BertForQuestionAnswering.from_pretrained(model_name).to(device)
tokenizer = BertTokenizer.from_pretrained(model_name)



model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
from transformers import AdamW
optim = AdamW(model.parameters(), lr=5e-5)

epochs = 3
device = torch.device('cuda:0' if torch.cuda.is_available()
                      else 'cpu')

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
# whole_train_eval_time = time.time()

# train_losses = []
# val_losses = []

# print_every = 1000

# for epoch in range(epochs):
#     epoch_time = time.time()
    
#     # Set model in train mode
#     model.train()
#     loss_of_epoch = 0
    
#     print("############Train############")
    
#     for batch_idx,batch in enumerate(train_loader): 
#         optim.zero_grad()
        
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         start_positions = batch['start_positions'].to(device)
#         end_positions = batch['end_positions'].to(device)
        
#         outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
#         loss = outputs[0]
#         # do a backwards pass 
#         loss.backward()
#         # update the weights
#         optim.step()
#         # Find the total loss
#         loss_of_epoch += loss.item()
        
#         if (batch_idx+1) % print_every == 0:
#             print("Batch {:} / {:}".format(batch_idx+1,len(train_loader)),"\nLoss:", round(loss.item(),1),"\n")
        
#     loss_of_epoch /= len(train_loader)
#     train_losses.append(loss_of_epoch)
    
#     ##########Evaluation##################
    
#     # Set model in evaluation mode
#     model.eval()
    
#     print("############Evaluate############")
    
#     loss_of_epoch = 0
    
#     for batch_idx,batch in enumerate(val_loader):
        
#         with torch.no_grad():
            
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             start_positions = batch['start_positions'].to(device)
#             end_positions = batch['end_positions'].to(device)
            
#             outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
#             loss = outputs[0]
#             # Find the total loss
#             loss_of_epoch += loss.item()
            
#         if (batch_idx+1) % print_every == 0:
#             print("Batch {:} / {:}".format(batch_idx+1,len(val_loader)),"\nLoss:", round(loss.item(),1),"\n")
    
#     loss_of_epoch /= len(val_loader)
#     val_losses.append(loss_of_epoch)
    
#     # Print each epoch's time and train/val loss 
    
#     print("\n-------Epoch ", epoch+1,
#           "-------"
#           "\nTraining Loss:", train_losses[-1],
#           "\nValidation Loss:", val_losses[-1],
#           "\nTime: ",(time.time() - epoch_time),
#           "\n-----------------------",
#           "\n\n")

# print("Total training and evaluation time: ", (time.time() - whole_train_eval_time))

In [ ]:
# Example Illustration
# Let's illustrate this with a simplified example:
# Start Logits: These are scores or logits corresponding to each token in the input sequence, indicating the likelihood that each token is the start of the answer span.
# End Logits: Similarly, these logits indicate the likelihood that each token is the end of the answer span
# Suppose the input sequence has 10 tokens.
# BERT predicts start_logits and end_logits for each token.
# The ground truth indicates that the answer span starts at index 3 and ends at index 5.
# For simplicity, assume hypothetical logits:

# start_logits = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# end_logits = [0.5, 0.4, 0.3, 0.2, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
# start_positions = 3 (index where answer span starts)
# end_positions = 5 (index where answer span ends)
# Calculation:

# start_loss = CrossEntropyLoss([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 3)
# end_loss = CrossEntropyLoss([0.5, 0.4, 0.3, 0.2, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6], 5)
# total_loss = start_loss + end_loss
# Conclusion
# In summary, the start and end logits are probabilities for each token indicating the likelihood that 
# it represents the start or end of the answer span. The loss function (CrossEntropyLoss in this case) quantifies the error between predicted logits and true positions. By minimizing this 
# loss during training, the model learns 
# to better predict accurate answer spans for given questions and contexts.

In [22]:
pip install peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [23]:
import torch
from torch import optim
from torch.utils.data import DataLoader, random_split
from transformers import BertTokenizerFast, BertForQuestionAnswering, BertConfig
from tqdm import tqdm
import timeit
import os

if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"

    LEARNING_RATE = 5e-5
    BATCH_SIZE = 4
    EPOCHS = 3  # Train for 3 epochs
    MODEL_PATH = "bert-base-uncased"
    MODEL_SAVE_PATH = "/kaggle/working/models/"

    # Assuming you have defined train_dataset and val_dataset
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True)
    
    val_loader = DataLoader(dataset=val_dataset,
                            batch_size=BATCH_SIZE,
                            shuffle=False)  # No need to shuffle validation data
    
    # Load tokenizer and model configuration from the pretrained model
    tokenizer = BertTokenizerFast.from_pretrained(MODEL_PATH)
    config = BertConfig.from_pretrained(MODEL_PATH)
    
    # Load model with the same configuration as the pretrained model
    model = BertForQuestionAnswering(config).to(device)
    
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

    # Training loop
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        print('-' * 20)
        
        # Training phase
        model.train()
        train_running_loss = 0
        for batch in tqdm(train_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                            start_positions=start_positions, end_positions=end_positions)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            train_running_loss += loss.item()

        train_loss = train_running_loss / len(train_loader)
        print(f"Train Loss: {train_loss:.4f}")

        # Validation phase
        model.eval()
        val_running_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                start_positions = batch['start_positions'].to(device)
                end_positions = batch['end_positions'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                                start_positions=start_positions, end_positions=end_positions)

                val_running_loss += outputs.loss.item()

        val_loss = val_running_loss / len(val_loader)
        print(f"Validation Loss: {val_loss:.4f}")

        print('-' * 20)

    # Save the fine-tuned model and tokenizer
    model.save_pretrained(MODEL_SAVE_PATH)
    tokenizer.save_pretrained(MODEL_SAVE_PATH)

    # Verify saved files
    print(f"Saved files: {os.listdir(MODEL_SAVE_PATH)}")


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch 1/3
--------------------


100%|██████████| 2713/2713 [10:42<00:00,  4.22it/s]


Train Loss: 4.3377


100%|██████████| 635/635 [00:34<00:00, 18.24it/s]


...

Validation Loss: 4.6373
--------------------
Epoch 3/3
--------------------


100%|██████████| 2713/2713 [10:42<00:00,  4.22it/s]


Train Loss: 3.6193


100%|██████████| 635/635 [00:34<00:00, 18.24it/s]


Validation Loss: 4.5364
--------------------
Saved files: ['vocab.txt', 'tokenizer_config.json', 'config.json', 'special_tokens_map.json', 'tokenizer.json', 'model.safetensors']


In [77]:
# Define the path to the SQuAD 2.0 training data
path = "/kaggle/input/squad-dataset-mee/train-v2.0.json"

# Load and preprocess the SQuAD 2.0 data
def load_squad_data(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    texts = []
    questions = []
    answers = []

    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    texts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return texts[10850:20000], questions[10850:20000], answers[10850:20000]

# Preprocess the data to find answer start and end positions
train_texts, train_queries, train_answers = load_squad_data(path)
for answer, text in zip(train_answers, train_texts):
    real_answer = answer['text']
    start_idx = answer['answer_start']
    # Get the real end index
    end_idx = start_idx + len(real_answer)

    # Deal with the problem of 1 or 2 more characters
    if text[start_idx:end_idx] == real_answer:
        answer['answer_end'] = end_idx
    # When the real answer is more by one character
    elif text[start_idx-1:end_idx-1] == real_answer:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1
    # When the real answer is more by two characters
    elif text[start_idx-2:end_idx-2] == real_answer:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2

train_encodings = tokenizer(train_texts, train_queries, truncation=True, padding = True)
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    
    count = 0
    
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))
        
        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
            
        # if end position is None, the 'char_to_token' function points to the space after the correct token, so add - 1
        if end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - 1)
            
            # if end position is still None the answer passage has been truncated
            if end_positions[-1] is None:
                count += 1
                end_positions[-1] = tokenizer.model_max_length
    print(count)
    
    # Update the data in dictionary
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)
tes_dataset = SquadDataset(train_encodings)
test_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

0
0


In [79]:
# Testing the model
from sklearn.metrics import f1_score
preds = []
true = []
with torch.no_grad():
    for idx, sample in enumerate(tqdm(test_loader)):
        input_ids = sample['input_ids'].to(device)
        attention_mask = sample['attention_mask'].to(device)
        start_positions = sample['start_positions']
        end_positions = sample['end_positions']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        start_pred = torch.argmax(outputs['start_logits'], dim=1).cpu().detach()
        end_pred = torch.argmax(outputs['end_logits'], dim=1).cpu().detach()

        preds.extend([[int(i), int(j)] for i, j in zip(start_pred, end_pred)])
        true.extend([[int(i), int(j)] for i, j in zip(start_positions, end_positions)])

preds = [item for sublist in preds for item in sublist]
true = [item for sublist in true for item in sublist]
f1_value = f1_score(true, preds, average="macro")
print(f"F1 Score: {f1_value}")

100%|██████████| 1144/1144 [02:27<00:00,  7.76it/s]

F1 Score: 0.016194817412470978


In [24]:
from torch.utils.data import DataLoader, Dataset

In [54]:

MODEL_SAVE_PATH = "/kaggle/working/models/"

# Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained(MODEL_SAVE_PATH)
model = BertForQuestionAnswering.from_pretrained(MODEL_SAVE_PATH).to(device)

In [53]:
model = BertForQuestionAnswering.from_pretrained(MODEL_SAVE_PATH)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [55]:
def predict(question, context):
    # Tokenize inputs
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    
    # Extract input IDs and attention mask
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    
    # Perform the model forward pass
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits
    
    # Get the most likely start and end positions
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits) + 1  # Add 1 to include the end token
    
    # Retrieve the answer span and convert tokens back to string
    answer = tokenizer.decode(input_ids[0][start_index:end_index])
    return answer


In [57]:
Context = "William Shakespeare was an English playwright, poet, and actor, widely regarded as the greatest writer in the English language."
Question = "Who is considered the greatest writer in the English language?"
print(predict(Question,Context))

william shakespeare was an english playwright


In [60]:
Context = "The moon orbits the Earth approximately once every 27.3 days."
Question = "How long does it take for the moon to orbit the Earth?"
print(predict(Question,Context))

orbits the earth approximately once every 27. 3 days.


In [61]:
Context = "The Great Barrier Reef is the world's largest coral reef system composed of over 2,900 individual reefs."
Question = "What is the Great Barrier Reef?"
print(predict(Question,Context))

barrier reef is the world's largest coral reef system composed of over 2, 900 individual reefs


In [63]:
Context = "The Declaration of Independence was adopted on July 4, 1776, declaring the independence of the United States from Great Britain."
Question = "On what date was the Declaration of Independence adopted?"
print(predict(Question,Context))

july 4,


In [64]:
Context = "The theory of general relativity, formulated by Albert Einstein, revolutionized our understanding of gravity."
Question = "What scientific theory did Albert Einstein formulate?"
print(predict(Question,Context))

gravity.


In [66]:
a = "The Hubble Space Telescope, launched into orbit in 1990, has provided unprecedented views of distant galaxies."
b = "When was the Hubble Space Telescope launched?"
print(predict(a,b))

1990, has provided unprecedented views of distant galaxies


In [69]:
a = "The Renaissance, a period of cultural and intellectual flourishing in Europe, spanned from the 14th to the 17th century."
b = "What historical period is known for its cultural and intellectual flourishing in Europe?"
print(predict(a,b))

In [70]:
a = "The Industrial Revolution, beginning in the late 18th century, brought about significant social and economic changes in Europe."
b = "What historical event led to major social and economic transformations in Europe during the late 18th century?"
print(predict(a,b))

18th century, brought about significant social and economic changes in europe. [SEP] what historical event led to major social and economic transformations in europe during the late 18th


In [72]:
a = " Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "
b = "When did Beyonce start becoming popular?"
print(predict(a,b))

dangerously in love ( 2003 ), which established her as a solo artist worldwide, earned five


In [74]:
a = "The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ('Norman' comes from 'Norseman') raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries."
b =  "In what country is Normandy located?"
print(predict(a,b))